In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import math

C:\Users\Sean Laney\Anaconda2\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# PrivateTest 3589, PublicTest 3589, Training 28709, 35887 total
def csv_reading(f_name):
    with open(f_name, 'r') as data:
        temp = pd.read_csv(data)
    temp = temp.values.tolist()
    for i in range(len(temp)):
        temp[i][1] = temp[i][1].split(' ')
    images = [item[1] for item in temp]
    for i in range(len(images)):
        images[i] = np.asarray([int(x) for x in images[i]])
        images[i] = images[i].reshape([48, 48, 1])
    labels = np.asarray([item[0] for item in temp])   
    return np.asarray(images), labels

In [3]:
images, labels = csv_reading('fer2013.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'fer2013.csv'

In [6]:
# cv2.imwrite('test1.jpg', images[2])

In [7]:
train_img = images[:32298]
test_img = images[32298:]
train_labels = labels[:32298]
test_labels = labels[32298:]


In [8]:
train_tensor = tf.convert_to_tensor(train_img, dtype=tf.float32)
labels_tensor = tf.convert_to_tensor(train_labels, dtype=tf.int64)

train_slices = tf.data.Dataset.from_tensor_slices(train_tensor)
labels_slices = tf.data.Dataset.from_tensor_slices(labels_tensor)

batch_size = 128
train_batch = train_slices.batch(batch_size).repeat()
labels_batch = labels_slices.batch(batch_size).repeat()

train_iter = train_batch.make_initializable_iterator()
labels_iter = labels_batch.make_initializable_iterator()

next_train = train_iter.get_next()
next_label = labels_iter.get_next()

In [9]:
input_img = tf.placeholder(tf.float32, shape=(None, 48, 48, 1))
input_labels = tf.placeholder(tf.int64, shape=None)

In [10]:
regularizer = tf.contrib.layers.l2_regularizer(scale=1e-7)

In [11]:
def cnn_fc_model(input_img):
    with tf.variable_scope('conv1'):
        conv1 = tf.layers.conv2d(input_img, 64, (3, 3), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv1', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv1 = tf.layers.batch_normalization(conv1, training=False, name='BN1', reuse=False)
        conv1 = tf.nn.relu(conv1, name='relu1')
        conv1 = tf.nn.dropout(conv1, 0.5, name='dropout1')       
        conv1 = tf.layers.max_pooling2d(conv1, (2, 2), (1, 1), padding='same', name='max_p1')
    with tf.variable_scope('conv2'):
        conv2 = tf.layers.conv2d(conv1, 128, (5, 5), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv2', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv2 = tf.layers.batch_normalization(conv2, training=False, name='BN2', reuse=False)
        conv2 = tf.nn.relu(conv2, name='relu2')
        conv2 = tf.nn.dropout(conv2, 0.5, name='dropout2')       
        conv2 = tf.layers.max_pooling2d(conv2, (2, 2), (2, 2), padding='same', name='max_p2')
    with tf.variable_scope('conv3'):
        conv3 = tf.layers.conv2d(conv2, 512, (3, 3), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv3', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv3 = tf.layers.batch_normalization(conv3, training=False, name='BN3', reuse=False)
        conv3 = tf.nn.relu(conv3, name='relu3')
        conv3 = tf.nn.dropout(conv3, 0.5, name='dropout3')       
        conv3 = tf.layers.max_pooling2d(conv3, (2, 2), (2, 2), padding='same', name='max_p3')
    with tf.variable_scope('conv4'):
        conv4 = tf.layers.conv2d(conv3, 512, (3, 3), 
                                 strides=(1, 1), 
                                 padding='same', 
                                 activation=None, 
                                 name='conv4', 
                                 reuse=False,
                                 kernel_regularizer=regularizer)
        
        conv4 = tf.layers.batch_normalization(conv4, training=False, name='BN4', reuse=False)
        conv4 = tf.nn.relu(conv4, name='relu4')
        conv4 = tf.nn.dropout(conv4, 0.5, name='dropout4')       
        conv4 = tf.layers.max_pooling2d(conv4, (12, 12), (1, 1), padding='valid', name='max_p4') 
        conv4 = tf.layers.Flatten()(conv4)
    with tf.variable_scope('fc1'):
        fc1 = tf.contrib.layers.fully_connected(conv4, 256, activation_fn=None, weights_regularizer=regularizer)
        fc1 = tf.layers.batch_normalization(fc1, training=False, name='BN5', reuse=False)
        fc1 = tf.nn.dropout(fc1, 0.5, name='dropout5')
        fc1 = tf.nn.relu(fc1, name='relu5')
    with tf.variable_scope('fc2'):
        fc2 = tf.contrib.layers.fully_connected(fc1, 512, activation_fn=None, weights_regularizer=regularizer)
        fc2 = tf.layers.batch_normalization(fc2, training=False, name='BN6', reuse=False)
        fc2 = tf.nn.dropout(fc2, 0.5, name='dropout6')
        fc2 = tf.nn.relu(fc2, name='relu6')
    result = tf.contrib.layers.fully_connected(fc2, 7, activation_fn=None)
    return result

In [12]:
# tf.reset_default_graph()
result_logits = cnn_fc_model(input_img)

In [13]:
softmax_losses = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=input_labels, logits=result_logits))
softmax = tf.nn.softmax(result_logits)

adamoptimizer = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-08)
train_op = adamoptimizer.minimize(softmax_losses)

In [14]:
def accuracy_compute(softmax_result, labels):
    prediction = tf.argmax(softmax_result, axis=1)
    correct = tf.equal(prediction, labels)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    return accuracy  

In [15]:
accuracy = accuracy_compute(softmax, input_labels)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
sess.run([train_iter.initializer, labels_iter.initializer])

In [15]:
# for epoch in range(1):  
    
for step in range(1000):
# for step in range(math.ceil(len(train_img)/batch_size)):
    temp_input = sess.run(next_train)
    temp_label = sess.run(next_label)
    _, temp_loss, temp_accuracy = sess.run([train_op, softmax_losses, accuracy], 
                                           feed_dict={input_img: temp_input, input_labels: temp_label})
    # sess.run(train_op, feed_dict={input_img: temp_input, input_labels: temp_label})
    print('step ', step+1, '  loss is  ', temp_loss, '  accuracy is  ', temp_accuracy)
    # print('step ', step+1)

InvalidArgumentError: assertion failed: [] [Condition x == y did not hold element-wise:] [x (SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [128] [y (SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [128 1 1]
	 [[Node: SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert = Assert[T=[DT_STRING, DT_STRING, DT_STRING, DT_INT32, DT_STRING, DT_INT32], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](SparseSoftmaxCrossEntropyWithLogits/assert_equal/All, SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert/data_0, SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert/data_1, SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert/data_2, SparseSoftmaxCrossEntropyWithLogits/Shape_1, SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert/data_4, SparseSoftmaxCrossEntropyWithLogits/strided_slice)]]

Caused by op 'SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert', defined at:
  File "/usr/lib/python3.6/runpy

In [19]:
current_loss, current_accuracy = sess.run([softmax_losses, accuracy], 
                                          feed_dict={input_img: test_img, input_labels: test_labels})
print('the current loss on test set is ', current_loss, '  accuracy is  ', current_accuracy)

the current loss on test set is  1.8547509   accuracy is   0.2398997


In [ ]:
# 1000